In [1]:
from __future__ import print_function
import json
import numpy as np


In [6]:
with open("songData.json") as f:
    documents = [(x['name'], x['category'], x['text'])
                 for x in json.loads(f.readlines()[0])
                 if len(x['text'].split()) > 50]

AttributeError: '_io.TextIOWrapper' object has no attribute 'size'